# Breast Cancer Dataset 
The Dataset is obtained from https://www.kaggle.com/uciml/breast-cancer-wisconsin-data/code 
My aim is to find the model which detects the least number of Malignant diagnosis as Benign. 
Feature Selection is first done using ANOVA (Analysis of Variance) F-test.
Minmax scalar is used.
I have tested Logistic Regression and KNNs to find the best model.
SMOTE can be used to tackle the imbalanced classes problem.
The aim is to find the most relavent Features. 
It can be observed even by using 1 feature 'concave points_worst', top predictor nearly 90% accuracy can be obtained. 


In [1]:
import pandas as pd


In [2]:
cancer_df = pd.read_csv('../input/breast-cancer-wisconsin-data/data.csv')
cancer_df.drop('Unnamed: 32',axis='columns',inplace=True)
cancer_df['diagnosis'] = cancer_df['diagnosis'].replace(['M','B'],[1,0])

In [3]:
benign = len(cancer_df[cancer_df['diagnosis'] == 0])
malignant = len(cancer_df[cancer_df['diagnosis'] == 1])
s = {'malignant':malignant,'beign':benign}
pd.Series(s)

malignant    212
beign        357
dtype: int64

In [4]:
X = cancer_df.drop('diagnosis',axis='columns')
y = cancer_df['diagnosis']

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from imblearn.over_sampling import SMOTE

X = cancer_df.drop('diagnosis',axis='columns')
y = cancer_df['diagnosis']
test = SelectKBest(score_func=f_classif)
fit = test.fit(X, y)
scores = fit.scores_
indices = (-scores).argsort()
feature_cols = list(X.columns)
feature_cols = [feature_cols[i] for i in indices]   # Feature columns arranged in the order of Importance
print('Features in the order of importance\n')
print(feature_cols)
scores = fit.scores_
scaler = MinMaxScaler()
for i in range(1,32):
    print('\nwith top {} features\n'.format(i))
    X_selected_features = X[feature_cols[0:i]]
    X_train, X_test, y_train, y_test = train_test_split(X_selected_features, y, random_state=0)
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    logistic = LogisticRegression(penalty='l2')
    logistic.fit(X_train, y_train)
    logistic.fit(X_train, y_train)
    knn = KNeighborsClassifier(n_neighbors=5,weights='uniform')
    knn.fit(X_train, y_train)
    print('***Using logistic regression***')
    print('Train accuracy {:.3f}'.format(logistic.score(X_train,y_train)))
    print('Test accuracy {:.3f}'.format(logistic.score(X_test,y_test)))
    print(confusion_matrix(y_test, logistic.predict(X_test)))
    print('\n***Using Knn regression***')
    print('Train accuracy {:.3f}'.format(knn.score(X_train,y_train)))
    print('Test accuracy {:.3f}'.format(knn.score(X_test,y_test)))
    print(confusion_matrix(y_test, knn.predict(X_test)))
    oversample = SMOTE(random_state=0)
    X_smote_train, y_smote_train = oversample.fit_resample(X_train, y_train)
    logistic.fit(X_smote_train, y_smote_train)
    print('\n***Using Smote & logistic regression***')
    print('Train accuracy {:.3f}'.format(logistic.score(X_smote_train,y_smote_train)))
    print('Test accuracy {:.3f}'.format(logistic.score(X_test,y_test)))
    print(confusion_matrix(y_test, logistic.predict(X_test)))

Features in the order of importance

['concave points_worst', 'perimeter_worst', 'concave points_mean', 'radius_worst', 'perimeter_mean', 'area_worst', 'radius_mean', 'area_mean', 'concavity_mean', 'concavity_worst', 'compactness_mean', 'compactness_worst', 'radius_se', 'perimeter_se', 'area_se', 'texture_worst', 'smoothness_worst', 'symmetry_worst', 'texture_mean', 'concave points_se', 'smoothness_mean', 'symmetry_mean', 'fractal_dimension_worst', 'compactness_se', 'concavity_se', 'fractal_dimension_se', 'smoothness_se', 'id', 'fractal_dimension_mean', 'texture_se', 'symmetry_se']

with top 1 features

***Using logistic regression***
Train accuracy 0.918
Test accuracy 0.909
[[87  3]
 [10 43]]

***Using Knn regression***
Train accuracy 0.908
Test accuracy 0.902
[[84  6]
 [ 8 45]]

***Using Smote & logistic regression***
Train accuracy 0.901
Test accuracy 0.895
[[82  8]
 [ 7 46]]

with top 2 features

***Using logistic regression***
Train accuracy 0.932
Test accuracy 0.944
[[89  1]
 [ 7

It can be Observed that using top 16 Features and KNN with 5 neighbours and uniform weights, False Negatives (Malignant as Benign)  are least.
Max Test set accuracy Observed is 97.9 % 
With True Positive Rate of 96.2 % in 2 cases
KNN with 5 nearest neighbours and top 16 features with uniform weights.
Logistic regression with L2 penalty, top 19 features and SMOTE over sampling to tackle the Class Imbalance. This model can be considered as the best model for this use case.
